In [1]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical

# Constants
image_size = (48, 48)  # Resize to 48x48
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Load and preprocess data from the dataset folder
def load_data_from_folder(dataset_folder):
    images = []
    labels = []
    label_map = {class_name: idx for idx, class_name in enumerate(class_names)}

    # Iterate through each folder (each class)
    for class_name in os.listdir(dataset_folder):
        class_folder = os.path.join(dataset_folder, class_name)
        
        if os.path.isdir(class_folder):  # Ensure we're dealing with a folder
            for filename in os.listdir(class_folder):
                file_path = os.path.join(class_folder, filename)
                
                try:
                    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Load image as grayscale
                    if img is not None:
                        img = cv2.resize(img, image_size)  # Resize image to (48, 48)
                        img_flat = img.flatten()  # Flatten image into a vector
                        images.append(img_flat)
                        labels.append(label_map[class_name])
                except Exception as e:
                    print(f"Error loading {file_path}: {e}")

    return np.array(images), np.array(labels)

# Load data from the 'dataset/' folder
dataset_folder = 'dataset'
X, y = load_data_from_folder(dataset_folder)

# Split the dataset into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data for the model and normalize
X_train = X_train.reshape((-1, 48, 48, 1)) / 255.0
X_val = X_val.reshape((-1, 48, 48, 1)) / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, len(class_names))
y_val = to_categorical(y_val, len(class_names))

# Balance classes
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(np.argmax(y_train, axis=1)),
    y=np.argmax(y_train, axis=1)
)
class_weights = dict(enumerate(class_weights))

# Define the model architecture
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer='l2', input_shape=(48, 48, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer='l2'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer='l2'),
    Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping, model checkpoint, and TensorBoard
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy')
tensorboard = TensorBoard(log_dir='logs', histogram_freq=1)

# Fit the model with data augmentation and class weights
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
model.fit(datagen.flow(X_train, y_train, batch_size=8),
          epochs=30,
          validation_data=(X_val, y_val),
          class_weight=class_weights,
          callbacks=[early_stopping, checkpoint, tensorboard])

Epoch 1/30
2871/2871 [==============================] - 245s 85ms/step - loss: 1.9765 - accuracy: 0.0942 - val_loss: 1.9417 - val_accuracy: 0.1701
Epoch 2/30
2871/2871 [==============================] - 236s 82ms/step - loss: 1.9465 - accuracy: 0.1350 - val_loss: 1.9484 - val_accuracy: 0.0160
Epoch 3/30
2871/2871 [==============================] - 201s 70ms/step - loss: 1.9464 - accuracy: 0.0892 - val_loss: 1.9425 - val_accuracy: 0.2567
Epoch 4/30
2871/2871 [==============================] - 300s 105ms/step - loss: 1.9464 - accuracy: 0.1854 - val_loss: 1.9488 - val_accuracy: 0.1459
